In [12]:
import h5py
import hdf5plugin
import io
import numpy as np

In [2]:
hdf5 = h5py.File('small.mzMLb', 'r+')

In [3]:
hdf5.keys()

<KeysViewHDF5 ['chromatogram_MS_1000515_float32', 'chromatogram_MS_1000595_float32', 'mzML', 'mzML_chromatogramIndex', 'mzML_chromatogramIndex_idRef', 'mzML_spectrumIndex', 'mzML_spectrumIndex_idRef', 'spectrum_MS_1000514_float64', 'spectrum_MS_1000515_float32']>

In [5]:
hdf5['mzML']

<HDF5 dataset "mzML": shape (287267,), type "|i1">

In [8]:
class HDF5ByteBuffer(io.RawIOBase):
    def __init__(self, buffer, offset=None):
        if offset is None:
            offset = 0
        self.buffer = buffer
        self.offset = offset
        self.size = self.buffer.size
        self.mode = 'rb'

    def readable(self):
        return True

    def seekable(self):
        return True

    def isatty(self):
        return False

    def seek(self, offset, whence=0):
        if whence == io.SEEK_SET:
            self.offset = offset
        elif whence == io.SEEK_CUR:
            self.offset += offset
        elif whence == io.SEEK_END:
            self.offset = self.size - offset
        else:
            raise ValueError(f"Bad whence {whence}")
        return self.offset

    def tell(self):
        return self.offset

    def close(self):
        return

    @property
    def closed(self):
        return False

    def readinto(self, b):
        n = len(b)
        temp = self._read(n)
        m = len(temp)
        b[:m] = temp[:]
        return m

    def readall(self):
        return bytes(self._read(-1))

    def read(self, n=-1):
        return bytes(self._read(n))

    def write(self, b):
        raise ValueError("Read-only stream")

    def _read(self, n=-1):
        if n == -1:
            n = self.size + 1
        dat = bytearray(np.array(self.buffer[self.offset:self.offset + n]))
        self.offset += n
        return dat


In [17]:
mzxml = io.BufferedReader(HDF5ByteBuffer(hdf5['mzML']))

array([ 60,  63, 120, 109, 108,  32, 118, 101, 114, 115, 105, 111, 110,
        61,  39], dtype=int8)

In [13]:
mzxml.read()

b'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<mzML xmlns="http://psi.hupo.org/ms/mzml" version="1.1.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://psi.hupo.org/ms/mzml http://psidev.info/files/ms/mzML/xsd/mzML1.1.0.xsd">\n  <cvList count="2">\n    <cv fullName="PSI-MS" URI="https://raw.githubusercontent.com/HUPO-PSI/psi-ms-CV/master/psi-ms.obo" version="4.1.62" id="PSI-MS"/>\n    <cv fullName="UNIT-ONTOLOGY" URI="http://ontologies.berkeleybop.org/uo.obo" version="releases/2020-03-10" id="UO"/>\n  </cvList>\n  <fileDescription>\n    <fileContent>\n      <cvParam cvRef="PSI-MS" accession="MS:1000579" name="MS1 spectrum" value=""/>\n      <cvParam cvRef="PSI-MS" accession="MS:1000580" name="MSn spectrum" value=""/>\n    </fileContent>\n    <sourceFileList count="1">\n      <sourceFile location="C:\\Users\\Joshua\\Dev\\ms_deisotope\\ms_deisotope\\test\\test_data" name="small.RAW" id="small.RAW">\n        <cvParam cvRef="PSI-MS" accession="MS:1000569" n